In [1]:
from tagger.core.mongo.models.order import Order
import pandas as pd
import simplejson
from datetime import datetime, timedelta


In [45]:
def get_data(d_from, d_to):
    orders = Order.objects(orderstatus__nin=["CREATED", "RECREATED", "PAYMENT_PENDING"], user__mobile__nin=["01028861089", "01097711882", "01077591771"], created__gte=d_from, created__lt=d_to)
#     print(d_from, d_to, [x for x in orders][0].id, [x for x in orders][-1].id)
    cancel_orders = [x for x in orders if "CANCEL" in x.orderstatus]
    timedeal_orders = orders.filter(ordertype="TIMEDEAL_ORDER")
    normal_orders = orders.filter(ordertype="NORMAL_ORDER")
#     df = pd.DataFrame.from_dict([x.to_mongo() for x in orders])
    count = len(orders)
    cancel_count = len(cancel_orders)
    cancel_rate = len(cancel_orders) / len(orders) if len(orders) != 0 else None
    avg_price = sum([x.totalprice for x in orders]) / len(orders) if len(orders) != 0 else None
    nominal_total = sum([x.totalprice for x in orders])
    non_cancel_total = sum([x.totalprice for x in orders if "CANCEL" not in x.orderstatus])

    timedeal_nominal_total = sum([x.totalprice for x in orders if "TIMEDEAL" in x.ordertype])
    timedeal_noncanc_total = sum([x.totalprice for x in orders if "CANCEL" not in x.orderstatus and "TIMEDEAL" in x.ordertype])
    normal_nominal_total = sum([x.totalprice for x in orders if "TIMEDEAL" not in x.ordertype])
    normal_noncanc_total = sum([x.totalprice for x in orders if "CANCEL" not in x.orderstatus and "TIMEDEAL" not in x.ordertype])
    
    timedeal_count = len(timedeal_orders)
    timedeal_cancel_count = len([x for x in timedeal_orders if "CANCEL" in x.orderstatus])
    timedeal_cancel_rate = len([x for x in timedeal_orders if "CANCEL" in x.orderstatus])  / len(timedeal_orders) if len(timedeal_orders) != 0 else None
    timedeal_avg_price = sum([x.totalprice for x in timedeal_orders]) / len(timedeal_orders) if len(timedeal_orders) != 0 else None

    normal_count = len(normal_orders)
    normal_cancel_count = len([x for x in normal_orders if "CANCEL" in x.orderstatus])
    normal_cancel_rate = len([x for x in normal_orders if "CANCEL" in x.orderstatus])  / len(normal_orders) if len(normal_orders) != 0 else None
    normal_avg_price = sum([x.totalprice for x in normal_orders]) / len(normal_orders) if len(normal_orders) != 0 else None
    normal_order_avg_price = sum([x.totalprice for x in normal_orders]) / sum([len(x.orders) for x in normal_orders]) if sum([len(x.orders) for x in normal_orders]) != 0 else None
    return {
        "count": count,
        "cancel_count": cancel_count,
        "cancel_rate": cancel_rate,
        "avg_price": avg_price,
        "nominal_total": nominal_total,        
        "non_cancel_total": non_cancel_total,        
        "timedeal_count": timedeal_count,
        "timedeal_cancel_count": timedeal_cancel_count,
        "timedeal_cancel_rate": timedeal_cancel_rate,
        "timedeal_avg_price": timedeal_avg_price,
        "normal_count": normal_count,
        "normal_cancel_count": normal_cancel_count,
        "normal_cancel_rate": normal_cancel_rate,
        "normal_avg_price": normal_avg_price,
        "normal_order_avg_price": normal_order_avg_price,
        "timedeal_nominal_total": timedeal_nominal_total,
        "timedeal_noncanc_total": timedeal_noncanc_total,
        "normal_nominal_total": normal_nominal_total,
        "normal_noncanc_total": normal_noncanc_total,
    }, {
        "count": count,
        "nominal_total": nominal_total,        
    }
    


In [46]:
def get_row(d_from, d_to):
    data, _ = get_data(d_from, d_to)
    return {"week": d_from, **data}

In [47]:
rows = [
    get_row(datetime(2021,8,31,15), datetime(2021,9,30,15)),
    get_row(datetime(2021,9,30,15), datetime(2021,10,31,15)),
]

# rows = [
#     get_row(datetime(2021,9,4,15), datetime(2021,9,11,15)),
#     get_row(datetime(2021,9,11,15), datetime(2021,9,18,15)),
#     get_row(datetime(2021,9,18,15), datetime(2021,9,25,15)),
#     get_row(datetime(2021,9,25,15), datetime(2021,10,2,15)),
#     get_row(datetime(2021,10,2,15), datetime(2021,10,9,15)),
#     get_row(datetime(2021,10,9,15), datetime(2021,10,16,15)),
#     get_row(datetime(2021,10,16,15), datetime(2021,10,23,15)),
#     get_row(datetime(2021,10,23,15), datetime(2021,10,30,15)),
# ]
# rows  =[]
# start = datetime(2021, 9, 3, 15)
# while start < datetime(2021, 11, 1):
#     rows.append(
#         get_row(start, start + timedelta(days=1))
#     )
#     start = start + timedelta(days=1)

In [48]:
pd.DataFrame(rows).to_clipboard()

In [13]:
pd.DataFrame(rows)["cancel_count"].to_clipboard()

datetime.datetime(2021, 10, 31, 15, 0)

In [90]:
all_orders = Order.objects(orderstatus__nin=["CREATED", "RECREATED", "PAYMENT_PENDING"], user__mobile__nin=["01028861089", "01097711882", "01077591771"])

In [91]:
timedeal_orders = all_orders.filter(ordertype="TIMEDEAL_ORDER")

In [93]:
td_items = [x.orders[0].alloffproduct for x in timedeal_orders]

In [95]:
brands = {}
for item in td_items:
    if item.brand.keyname in brands:
        brands[item.brand.keyname] += [item]
    else:
        brands[item.brand.keyname] = [item]        

In [107]:
brand_rows = []
for keyname in brands.keys():
    items = brands[keyname]
    count = len(items)
    price_sum = sum([x.discountedprice for x in items])
    brand_rows += [{"brand": keyname, "count": count, "total": price_sum, "avg_price": int(price_sum/count)}]

In [108]:
pd.DataFrame(brand_rows)

,brand,count,total,avg_price
0,MINE,14,2584330,184595
1,THURSDAYISLAND,1,59800,59800
2,SYSTEM,32,3730100,116565
3,SJSJ,15,1415820,94388
4,TIME,8,1106550,138318
5,DEWL,7,819870,117124
6,CCCOLLECT,6,622400,103733
7,LATT,2,139120,69560
8,OBZEE,8,1942590,242823
9,O2ND,1,72380,72380


In [41]:
orders = Order.objects(orderstatus__nin=["CREATED", "RECREATED", "PAYMENT_PENDING"], user__mobile__nin=["01028861089", "01097711882", "01077591771"], created__lt=datetime(2021, 10, 31, 15))

In [42]:
usermap = {}
for order in orders:
    existing = usermap[order.user._id] if order.user._id in usermap else []
    usermap[order.user._id] = existing + [order]

In [43]:
usermap_data = []
for key, val in usermap.items():
    non_cancels = [x for x in val if "CANCEL" not in x.orderstatus]
    usermap_data.append({
        "nominal_count": len(val),
        "nominal_total": sum([x.totalprice for x in val]),
        "non_cancel_count": len(non_cancels),
        "non_cancel_total": sum([x.totalprice for x in non_cancels]),
        "userid": key,
    })

In [44]:
pd.DataFrame(usermap_data).to_clipboard()

In [54]:
for o in [x for x in Order.objects(orderstatus__nin=["CREATED", "RECREATED", "PAYMENT_PENDING"], user__mobile__nin=["01028861089", "01097711882", "01077591771"], created__gte=datetime(2021,10,16,15), created__lt=datetime(2021,10,23,15))]:
    print(o.totalprice, o.id)

97350 616afa8917fc2e5a1f9e8ba0
161100 616b1f5b17fc2e5a1f9e8bb7
292500 616b8a1717fc2e5a1f9e8bed
97350 616b8a2517fc2e5a1f9e8bee
129000 616bcbcc17fc2e5a1f9e8c69
129000 616bcc2c17fc2e5a1f9e8c6e
129000 616bcccd17fc2e5a1f9e8c79
97350 616bcdd517fc2e5a1f9e8c82
236000 616bcdf117fc2e5a1f9e8c84
236000 616bd12d17fc2e5a1f9e8c97
129000 616bf48117fc2e5a1f9e8d09
168000 616bf95e17fc2e5a1f9e8d16
166000 616bfc4c17fc2e5a1f9e8d24
236000 616c17b217fc2e5a1f9e8d8e
38700 616c1e9617fc2e5a1f9e8da5
106000 616c2d1317fc2e5a1f9e8dd5
159200 616c316c17fc2e5a1f9e8e05
305800 616c331a17fc2e5a1f9e8e0b
131670 616c3c7c17fc2e5a1f9e8e1f
637030 616c3cbf17fc2e5a1f9e8e22
131670 616c6ab017fc2e5a1f9e8e42
131670 616ca14b17fc2e5a1f9e8ead
97350 616d47d317fc2e5a1f9e8fbe
82000 616d495d17fc2e5a1f9e8fc3
230670 616d560317fc2e5a1f9e8ffc
166350 616d631317fc2e5a1f9e9020
166350 616d707617fc2e5a1f9e907d
113700 616d712e17fc2e5a1f9e9085
113700 616d716017fc2e5a1f9e908a
1430000 616d717c17fc2e5a1f9e908d
298350 616d801417fc2e5a1f9e90f1
156000 616da2

In [25]:
usermap

{}